# Lab 2: Preprocessing

In this lab, we will have you look at some of the essential preprocessing steps that should be conducted before any analysis. So that you can get familiar with the impact of each step, we will have you apply them separately.


<div class="warning" style='background-color:#805AD5; color: #90EE90; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>⚠️ Preprocessing warnings ⚠️</b></p>
<p style='text-indent: 10px;'>
The steps of preprocessing you have seen in class are dependent on the analysis you wish to conduct.
In particular, there is no yet a clear consensus on the order in which some steps should be applied, although said order is known to impact subsequent analysis.
As an example, we will teach you how to put your subjects in a common space, the MNI space through a step of <b>normalization</b> (more on that later!), such that subjects can be compared at the group-level. Assume now that for the purpose of your analysis, you are interested only in a single subject, or that the method of choice should be at an individual level for your application. Clearly the normalization is superfluous in this case.
You should also know that most steps will have parameters to set. These parameters can be set at the population level (this is often the case). Such choice means the preprocessing won't be optimal for each subject.
<br>
</p></span>
</div>
<br>
<div class="warning" style='background-color:#90EE90; color: #805AD5; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>Preprocessing pipelines</b></p>
<p style='text-indent: 10px;'>
We will teach you how to perform each step individually, so that you get a precise understanding of what each step's purpose is. You should know nonetheless that software is available to perform these steps for you. FSL itself already provides such a pipeline, which is simply the steps you will see chained after each other, with the choice to disable or enable specific steps, but you can also consider for instance <a href="https://fmriprep.org/en/stable/">fMRIPrep</a> in the case of fMRI data for example. 
<br>
</p></span>
</div>
<br>
<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid #darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'>
<b>💡 Quality Control (QC): the 1 - 10 - 100 dollar rule 💡</b></p>
<p style='text-indent: 10px;'>
The 1-10-100 rule states that it takes 1 dollar to verify and correct data at the start, 10 dollars to identify and clean data after the fact and 100 dollars to correct a failure due to bad data.

In preprocessing this is especially true. It will take you much less effort to first look at your data, detect what might be wrong from the start and deal with it rather than apply everything blindly and notice after the fact that something went wrong. Always, ALWAYS look at your data before <u>anything and any analysis</u>. A surgeon always looks at the patient before operating, you should do the same: you are surgeons to your dataset, so please look at it carefully (it's craving for attention, the poor thing 💔).
These intermediary steps of controlling the health of your dataset are called quality control steps. You're doing exactly what you might expect to do: you check the quality of your data before and after a given step, to ensure that nothing went awry for example.
<br>
</p></span>
</div>

In [ ]:
# General purpose imports to handle paths, files etc
import os
import os.path as op
import glob
import pandas as pd
import numpy as np
import json


# Useful functions to define and import datasets from open neuro
import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt



def reset_overlays():
    """
    Clears view and completely remove visualization. All files opened in FSLeyes are closed.
    The view (along with any color map) is reset to the regular ortho panel.
    """
    l = frame.overlayList
    while(len(l)>0):
        del l[0]
    frame.removeViewPanel(frame.viewPanels[0])
    # Put back an ortho panel in our viz for future displays
    frame.addViewPanel(OrthoPanel)
    
def mkdir_no_exist(path):
    if not op.isdir(path):
        os.makedirs(path)
        
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)


def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def direct_file_download_open_neuro(file_list, file_types, dataset_id, dataset_version, save_dirs):
    # https://openneuro.org/crn/datasets/ds004226/snapshots/1.0.0/files/sub-001:sub-001_scans.tsv
    for i, n in enumerate(file_list):
        subject = n.split('_')[0]
        download_link = 'https://openneuro.org/crn/datasets/{}/snapshots/{}/files/{}:{}:{}'.format(dataset_id, dataset_version, subject, file_types[i],n)
        print('Attempting download from ', download_link)
        download_url(download_link, op.join(save_dirs[i], n))
        print('Ok')
        
def get_json_from_file(fname):
    f = open(fname)
    data = json.load(f)
    f.close()
    return data

## 0. Loading a dataset

We have not touched yet upon how you might load datasets. 

### BIDS standard
For this, we should first tell you more about a standard: the BIDS standard.
This one allows you to format a dataset such that other researchers in neuroimaging can reuse your data with the smallest overhead possible. It is a way to unify how files and acquisitions are organized in folders.

This unification comes with several advantages including many tools that ease our life.
Indeed, if your dataset is in such a format, it is very easy to conduct any analysis: your scripts will expect a specific structure, so you don't need to play a million times with paths for example!

A whole software ecosystem has evolved around this standard, including tools that enable us to load datasets that are BIDs compliant!

In [ ]:
dataset_fmap = 'ds004226'
subject_fmap = '001' 

# Download one subject's data from each dataset
bids_root = op.join(op.dirname(sample.data_path()), dataset_fmap)

mkdir_no_exist(bids_root)

In [ ]:
openneuro.download(dataset=dataset_fmap, target_dir=bids_root,
                   include=['sub-' + subject_fmap + '/anat/*',
                            'sub-' + subject_fmap + '/func/sub-001_task-sitrep_run-01_bold.nii.gz', 
                            'sub-' + subject_fmap + '/func/sub-001_task-sitrep_run-01_bold.json'], max_concurrent_downloads=1)

So we have this nice message of download being finished, all is good right?
Well no! Have a look at the terminal in which you launched this notebook. You will see an output that looks like this:
<img src="imgs/download_picture.png"/>

What this tells you is that the download is, actually, still undergoing. Be mindful of this when downloading a dataset: you should avoid opening a file until it is fully downloaded, otherwise you have a high chance of corrupting it!

Notice as well that the download rate might be a bit slow :( It's a shame. 
But we can circumvent it by downloading ourselves the files directly. We provide you with a function to do precisely this (the not so nice thing is that you have to explicitly specify the files you want with this function). Here's how it would look like to load the above dataset:

In [ ]:
func_path = op.join(bids_root, 'sub-001', 'func')
anat_path = op.join(bids_root, 'sub-001', 'anat')
mkdir_no_exist(op.join(bids_root, 'sub-001'))
mkdir_no_exist(func_path)
mkdir_no_exist(anat_path)

direct_file_download_open_neuro(file_list=['sub-001_task-sitrep_run-01_bold.nii.gz', 
                                           'sub-001_task-sitrep_run-01_bold.json',
                                           'sub-001_T1w.nii.gz',
                                           'sub-001_T1w.json'], 
                                file_types=['func', 'func', 'anat', 'anat'], 
                                dataset_id=dataset_fmap, 
                                dataset_version='1.0.0', 
                                save_dirs=[func_path,
                                           func_path,
                                           anat_path,
                                           anat_path])

In [ ]:
mkdir_no_exist(op.join(bids_root, 'derivatives'))
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')
mkdir_no_exist(preproc_root)
mkdir_no_exist(op.join(preproc_root, 'sub-001'))
mkdir_no_exist(op.join(preproc_root, 'sub-001', 'anat'))
mkdir_no_exist(op.join(preproc_root, 'sub-001', 'func'))
mkdir_no_exist(op.join(preproc_root, 'sub-001', 'fmap'))

Regardless of your choice, you can skip ahead a bit and work on slice-timing correction in the meantime just to keep busy.

Once the download is done, we can have a look at the resulting folder structure:

In [ ]:
print_dir_tree(bids_root, max_depth=4)

This organization is typical of a BIDs dataset. Each subject's file is split between anatomical data and functional data. You are already a bit familiar with the .nii.gz file extension, but what might be the .json file? Well, let's open it to figure it out!

In [ ]:
data = get_json_from_file(op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold.json'))
data

This JSON is extremely important: it is what we call a JSON **sidecar**, and it holds precious acquisition informations! Based **only on the text printed above**, are you able to determine any or all of the following?
- [ ] TR?
- [ ] Modality of acquisition?
- [ ] How many Teslas the scanner was?

### Loading more datasets: how to

Great! Note that we've loaded only one subject and one file of each modality for the subject. You can have a look <a href="https://openneuro.org/datasets/ds004226/versions/1.0.0">here</a> for this dataset. As you can see, it is a big dataset; we've restricted our download to the bare minimum to spare your computer's disk space as much as possible.

Notice two things on the web page. The first is the dataset's accession number:
<img src="imgs/openneuro_access_nbr.png">
This number is the one we've put in our code earlier, to specify what dataset we wanted to load from:
```python
dataset_fmap = 'ds004226'
...
openneuro.download(dataset=dataset_fmap, ...)

```

Should you wish to download another openneuro dataset that piqued your interest, you'll simply need to change the above variable with the accession number of the dataset on the corresponding page!

Secondly, you can observe the entire folder structure, size and many other interest informations of this dataset by simply scrolling its page!
<img src="imgs/openneuro_full_view.png">
You can really decide whether a dataset is what you need this way, before burdening your connection with any heavy download :)

## 1. Anatomical preprocessing

Let's have a look at the nice anatomical we downloaded above.

In [ ]:
reset_overlays()
load(op.join(bids_root, 'sub-001', 'anat', 'sub-001_T1w.nii.gz'))

Take some time to explore the exquisite anatomy of the brain. Notice around the brain, the human skull. It is full of regions which show up **in this contrast** whiter than others. Based on what you might know from class about the T1 contrast, can you identify the different regions annotated below taken from another T1?

As a hint, think about white matter: it is composed of axons and myelin. Contrast with grey matter, which is comprised of soma. Based on your understanding:
<img src="imgs/annotated_regions.png">


- [ ] Tissues high in fat are bright in T1 contrast, which is why white matter is brighter than grey matter
- [ ] Tissues high in fibers are bright in T1 contrast, which is why white matter is brighter than grey matter
- [ ] Region 1 is likely high in fibers and might be tendons and ligaments, which are dense connective tissues full of fibers.
- [ ] Region 1 is likely high in fat, and is probably subcutaneous fat.
- [ ] Region 2 contains a mix of fat and water, hence the slightly darker color. Given its location, it is probably bone marrow.
- [ ] Region 2 contains a mix of fibers and water, hence the slightly darker color. Given its location, it is probably the dura mater, connective tissues that make up the outer-most layer of the meninges.
- [ ] Region 3 contains air, which is why we do not see it in T1.
- [ ] Region 3 contains mostly water, which appears dark in T1.


### Skull stripping

#### Preprocessing and BIDs
An important part of **anatomical** preprocessing is to remove the skull around the brain.
To adhere to the BIDs format, all modified files should be put in a new folder, called derivatives, such that you always have clean data in the source directory. The derivatives folder can be used for different preprocessing and treatments, each needing their own subfolders. In our case, we've created a single folder, preprocessed_data, hence the following structure:

In [ ]:
print_dir_tree(bids_root, max_depth=4)

#### Actual skull stripping

Perfect! Let's move on to actually extracting the brain! To make it easier for you to detect what was actually extracted, we will let the brain extraction proceed, using FSL's <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/BET/UserGuide">BET</a> (brain extraction tool) and show you the mask of the region determined by FSL as brain.

In [ ]:
preproc_root

In [ ]:
anatomical_path = op.join(bids_root, 'sub-001', 'anat', 'sub-001_T1w.nii.gz')
betted_brain_path = op.join(preproc_root, 'sub-001', 'anat', 'sub-001_T1w')
resulting_mask_path = op.join(preproc_root, 'sub-001', 'anat', 'sub-001_T1w_mask')
bet(anatomical_path, betted_brain_path, mask=resulting_mask_path)

In [ ]:
load(resulting_mask_path)

Is the mask nicely fitting around the brain? What you would like is that the mask is taking all parts of the brain and excluding the rest.
To answer this one, play with the mask's opacity in FSL eyes.<br>
*Hint: have a look at the frontal regionsm inspect as well the superior parietal regions*<br>
<br><br>
What you are doing here is simply **Quality Control** (QC). It is a crucial step that you should **NEVER** skip when dealing with data preprocessing. As all steps are dependent on the success of previous steps, always make sure that everything is performing properly before moving on!

#### Improving the fit
If you look a bit into bet's documentation, you'll quickly find that there are parameters with which you can play; robust brain centre estimation and fractional intensity threshold. To demonstrate the importance and impact of these parameters, let's use a robust brain center estimation.

In [ ]:
bet(anatomical_path, betted_brain_path, mask=resulting_mask_path, robust=True)

In [ ]:
reset_overlays()

load(op.join(bids_root, 'sub-001', 'anat', 'sub-001_T1w.nii.gz'))
load(resulting_mask_path)

How good is the mask now?

Is it perfect? (*Hint: have a look at voxels around 94-209-131*)

#### Hand corrections
If you really want good fit, you might want to resort to **hand correcting the mask**. 

FSLeyes readily allows you to do such things! While on FSLeyes, press **Alt + E** to open the editing interface.
<img src="imgs/editing_menu_fsl.png">
<center><i>FSLeyes editing menu</i></center>

We will work here on removing some unwanted voxels. Toggle the 'Select mode' first. This way, FSL will show us which voxels we currently have selected, before changing their value
<img src="imgs/selection_mode_toggle_fsl.png">
<center><i>Make sure to be in Select mode by clicking it</i></center>

Then let's pick the pencil tool, to select the voxels we want.
<img src="imgs/brush_tool.png">

Good, we're set and we can now select voxels. We'll try to select some **unwanted** voxels. Simply paint over them!
<img src="imgs/painted_voxels.png">
<center><i>Selected voxels are shown in purple</i></center>

Now, we are dealing with a mask. We thus want to put the value of our selection to **0**, so as to remove it from the mask. To do so, we must change the fill value to 0, and then click to replace our selection with the provided value:
<img src="imgs/paint_steps.png">
<center><i>The two steps to set selection to a specific fill value.</i></center>
<img src="imgs/mask_painted.png">
<center><i>Painting with zero a mask means we remove the painted voxels from the mask.</i></center>

It remains now to apply the mask to our anatomical data. This is fortunately something that you now know how to do from the previous lab! Fill in the next cell with the appropriate code **and make sure to save the masked brain in the proper directory**.

In [ ]:
anatomical_path

In [ ]:
# Fill me with the code to use your mask and apply it to the subject's anatomical data. 
# Save the result in the derivatives folder!!

## SOLUTION
# To solve this one, we can go two ways:
# 1- Load the volume in the notebook and mask it the "pythonic" way
# 2- Use flsmaths's function to do it natively

# For 1:
# - Load the mask with nibabel
# - Load the volume with nibabel
# - Set to zero anything outside the mask, the numpy way
# - Write the result with nibabel
import nibabel as nib
mask = nib.load(glob.glob(resulting_mask_path + '.nii*')[0])
img_to_mask = nib.load(glob.glob(anatomical_path)[0])

mask_data = mask.get_fdata()
img_data = img_to_mask.get_fdata() # Remember: a nifti is header + payload, so we must extract the payload
img_data[~( np.abs(mask_data - 1.0)< 10e-8)] = 0 # Here, we use the mask as a boolean and take all values where it is not True
masked_img = nib.Nifti1Image(img_data, img_to_mask.affine, img_to_mask.header)
# Save the image
nib.save(masked_img, 'masked_attemp')  


# For 2:
# If you look up fslmaths, you'll see that it has a -mas function. This function is used to apply a mask (!!) to an image.
# Exactly the one we need, don't you think?
# We do it this way:
fslmaths(anatomical_path).mas(resulting_mask_path).run('betted_img_example')

In [ ]:
load('betted_img_example')

### Tissue segmentation

For the purpose of analysis, it can be useful to separate the tissues into tissue classes; in particular extracting the white matter, grey matter and cerebrospinal fluid (abreviated as CSF) is very interesting in fMRI analysis. Consider for example an analysis that you wish to restrict to the somas of your neurons, would it make sense to conduct your analysis on the CSF ?

You'll find that the segmentation is not done on fMRI volumes; it is done on the anatomical and the resulting tissue masks are then used on the functional data. Can you imagine why this is the case?

Let's perform tissue segmentation. To do so, we'll use FSL's <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FAST">FAST</a> (FMRIB's Automated Segmentation Tool).

The underlying idea of FAST is to try and model each voxel's intensity as being a mixture between the different tissue types.
Pay attention in the documentation to the following line:
<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b></b></p>
<p style='text-indent: 10px;'>
    Before running FAST an image of a head should first be brain-extracted, using BET. The resulting brain-only image can then be fed into FAST.</p>
</span>
</div>

Based on this, **in the cell below choose which image should be used as fast_target, between the anatomical_path and the brain_extracted_path images**.


<div class="warning" style='background-color:orange; color: #112A46; border-left: solid red 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>🐞 Troubleshooting: FSL stopped responding </b></p>
<p style='text-indent: 10px;'>
    It is perfectly possible (even likely) that FSLeyes will stop responding over the course of this lab. This is perfectly normal! Simply wait for whichever function (such as FAST) to finish and it should start responding again, don't worry too quickly, be patient :)</p>
</span>
</div>

In [ ]:
anatomical_path = op.join(bids_root, 'sub-001', 'anat', 'sub-001_T1w.nii.gz')
bet_path = op.join(preproc_root, 'sub-001', 'anat', 'sub-001_T1w')

fast_target = bet_path # Replace with either anatomical_path or bet_path (note: you can try both and decide which is more reasonable!)

[os.remove(f) for f in glob.glob(op.join(preproc_root, 'sub-001', 'anat', '*fast*'))] # Just to clean the directory in between runs of the cell
segmentation_path = op.join(preproc_root, 'sub-001', 'anat', 'sub-001_T1w_fast')
fast(imgs=[fast_target], out=segmentation_path, n_classes=3)

Let's check the quality of the segmentation, shall we?
We want to extract 3 tissue types here: the white matter, the grey matter and the csf. How well did fast perform?

In [ ]:
reset_overlays()
load(bet_path)

If you look at the directories now, we have new files in our hierarchy:

In [ ]:
print_dir_tree(bids_root, max_depth=5)

The pve files correspond to our segmented tissues. We have exactly three files, because we set n_classes to 3 above:
```python
fast(..., n_classes=3)
```

Let's try to identify which segmentation is which tissue type in the brain. To do this, you'll have to visualize the tissues and decide for yourself:
- [ ] pve_0 is white matter, pve_1 is grey matter, pve_2 is CSF
- [ ] pve_0 is grey matter, pve_1 is white matter, pve_2 is CSF
- [ ] pve_0 is grey matter, pve_1 is CSF, pve_2 is white matter
- [ ] pve_0 is CSF, pve_1 is grey matter, pve_2 is white matter


To make it easier on you, we will display:

- pve_0 in <span style="color:red;">red</span>
- pve_1 in <span style="color:green;">green</span>
- pve_2 in <span style="color:blue;">blue</span>

In [ ]:
load(glob.glob(op.join(preproc_root, 'sub-001', 'anat','*pve_0*'))[0])
load(glob.glob(op.join(preproc_root, 'sub-001', 'anat','*pve_1*'))[0])
load(glob.glob(op.join(preproc_root, 'sub-001', 'anat','*pve_2*'))[0])
displayCtx.getOpts(overlayList[1]).cmap = 'Red'
displayCtx.getOpts(overlayList[2]).cmap = 'Green'
displayCtx.getOpts(overlayList[3]).cmap = 'Blue'

<div class="warning" style='background-color:#90EE90; color: #112A46; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>🏠 Tissues and contrast: Take home message 🏠</b></p>
<p style='text-indent: 10px;'>
    Tissues in T1 or T2 will show up in diffrent colour, dependent on their content. This is because their content affects their relaxation time and, in turn, the intensity captured during the acquisition. Here are different tissue types for both modalities (from <a href="https://www.researchgate.net/publication/324396120_Basic_MRI_for_the_liver_oncologists_and_surgeons">Vu, Lan N., John N. Morelli, and Janio Szklaruk. "Basic MRI for the liver oncologists and surgeons." Journal of hepatocellular carcinoma 5 (2017): 37.</a>):
    <table>
        <tr>
            <th>MR</th>
            <th>High signal (bright)</th>
            <th>Low signal (dark)</th>
        </tr>
        <tr>
            <th>T1</th>
            <th>Fat, melanin</th>
            <th>Iron</th>
        </tr>
        <tr>
            <th></th>
            <th>Blood</th>
            <th>Water</th>
        </tr>
        <tr>
            <th></th>
            <th>Proteinaceous fluid</th>
            <th>Air, bone</th>
        </tr>
        <tr>
            <th></th>
            <th>Paramagnetic substances</th>
            <th>Collagen</th>
        </tr>
        <tr>
            <th></th>
            <th>Chelated gadolinium contrast</th>
            <th>Most tumors</th>
        </tr>
        <tr>
            <th>T2</th>
            <th>Water</th>
            <th>Air</th>
        </tr>
        <tr>
            <th></th>
            <th>Edema</th>
            <th>Bone</th>
        </tr>
        <tr>
            <th></th>
            <th>Blood</th>
            <th>Hemosiderin, deoxyhemoglobin, methemoglobin</th>
        </tr>
    </table>
    Typical preprocessing steps of anatomical data starts by extracting the brain by removing skull tissues. This step can be conducted mostly automatically, but it is perfectly possible to manually correct the extracted brain, either to include more or less voxels when tweaking the parameters does not yield satisfactory results.
    The extracted brain can be segmented into different tissues. Using the difference in brightness due to contrast, we can separate the grey matter, the white matter and the CSF, which is useful for later analysis.</p>
</span>
</div>

## Normalization
Let's try to do something. We will load our anatomical, along with the MNI template you saw last week, overlayed on top of each other.


They are not very well aligned, but we can try to make them more aligned. Specifically, we would like to find a transformation such that we can align our anatomical to the MNI template. This is the so-called normalization step.
So we need two ingredients to do this:
- A way to compute the transformation from anatomical to the MNI template (this step is called registration)
- A reference image in the space of the MNI template (here, actually, this is the MNI template)

#### What transformation?

What should this transformation be?
It is a combination of translation, rotation, scaling and other possible modifications, applied on our anatomical, so that it ends up matched to the target (the MNI image). In essence, the transformation fully describes the process to align the two images!


#### Why the reference ?
<p>
Let's pause for a moment. If the transformation already encapsulates all that there is to know about how to transform the volume, why do we need a reference from the target space?

To answer this, let's think about what happens when moving the anatomical image, shall we?
We rotate it, translate it, maybe shear it, cool, we have an anatomical well registered. But there's still an issue.
<br><b>What is the resolution of our anatomical?</b><br>
If you remember, the anatomical has an exquisite spatial resolution, but it might not be exactly the same as the MNI template: what if you used the MNI with 0.5mm voxel size from last week for example? In this case, we have a mismatch in resolutions. Let's get concrete with an example with Ducky!
</p>

#### Ducky's sunglasses
<p>    
Imagine the following example: we have an image of a duck (Ducky!), and we want to align sunglasses to it. 
Notice that this is hard, algorithmically, right? Unlike the brain, there  are no landmarks to use such as white-matter to try and optimize some cost. Sunglasses could be worn on the beak, the head, even the neck: there are no rules to fashion!
Fortunately, we are told Ducky should wear the glasses on its beak like the cool kids and are even provided the transformation to do so, a combination of translation, rotation and scaling. Applying this transform to the sunglasses, we get the following:
    
<div>
<center>
<div style="display:inline-block;">
    <img src="imgs/ducky/ducky_alone.png" style="width: 200px; height:auto; border: blue 6px groove;"/>
    <p style="text-align:center;">Ducky (our reference!)</p>
</div>
<div style="display:inline-block;">
    <img src="imgs/ducky/sunglasses_alone.png"/ style="width: 150px; height:auto;border: green 6px groove;" />
    <p style="text-align:center;">The sunglasses (in their own space)</p>
</div>   
<div style="display:inline-block;">
    <img src="imgs/ducky/ducky_summary.png" style="width: 200px; height:auto; border: blue 6px groove;"/>
    <p style="text-align:center;">What the transformation will do</p>
</div>
    </center>
</div>

Okay, we do all this. We obtain this:
<img src="imgs/ducky/ducky_total_uncropped.png" style="width: 200px; height:auto; border: black 6px groove;"/>    

Now, why is Ducky so big?
The answer is simple: the sunglasses and Ducky did not have the same:
- Resolution (meaning the glasses can get blurry, think of putting a 144px resolution screenshot on a 4K resolution image!)
- Field of view (canvas size if you're thinking of Photoshop layers for example)

Here are the field of view of the two images, kept as is :
<img src="imgs/ducky/ducky_uncropped.png" style="width: 200px; height:auto; border: black 6px groove;"/>    
    
But we would like to put the sunglasses on ducky without increasing the size around ducky. <b>So let's use Ducky's picture's resolution and field of view to correct our transformed sunglasses' own resolution and field of view</b>.
In other words, we will interpolate the sunglasses to match our duck's resolution and we will crop its field of view (or pad it if necessary) to match perfectly our duck. Final result:
    <center><img src="imgs/ducky/ducky_complete.png" style="width: 200px; height:auto; border: black 6px groove;"/>    <p style="text-align:center;">Much better.</p>
</center>

</p>

#### Back to neurological data

We use exactly the same idea when applying transformations. It is for this reason that when applying a transformation in FSL, you will always need to pass a reference image of the space in which you want to end up. This way, FSL will adapt the field of view but also the resolution by interpolation. This interpolation parameter can be done through nearest neighbour, trilinear or sinc interpolation.


## Types of normalization

So, you now know that you need a transformation and a reference. Great. Now, the transformation you allow can be of two types: it can be linear, meaning whatever you apply will be the same across the entire image, or non linear, where each voxel gets a separate treatment

(ducky linear and non linear)



### Linear normalization

To perform linear normalization, the idea is simple. The transformation we want should be linear - ie, affine.
Such a matching is usually called in image processing <a href="https://en.wikipedia.org/wiki/Image_registration">image registration</a>. Here, we're dealing with 3D data, so the problem is a bit more complicated. Fortunately all of this has been coded by very smart people, and to our rescue comes a tool specifically to register volumes to each other: <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FLIRT/UserGuide">FLIRT</a>!
<br>
This tool can allow many registrations and is extremely powerful. In its most basic form, it expects:
- An input volume, the volume you want to register (Ducky's sunglasses)
- A reference volume, to which the input is registered (Ducky's body)
- An output volume, the result of the transformation (Ducky's sunglasses once they are on Ducky's beak)

The situation looks like this:
(Img of the patient, img of the reference, img of the two overlaid )


Here is how you can call it to register the patient's anatomical to some reference sitting in another space (here the MNI152 template):
```python
flirt()

```

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>💡 Pay attention! 💡</b></p>
<p style='text-indent: 10px;'>
    FLIRT also expects the anatomical to be skull stripped to maximize normalization.</p>
</span>
</div>

In [ ]:
anatomical_target = bet_path
reference = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))
result = op.join(preproc_root, 'sub-001', 'anat', 'sub-001_T1w_mni')
flirt(anatomical_target, reference, out=result)

Now comes the **QC** step: how well are we aligned with respect to our reference? Let's visualize it!

In [ ]:
reset_overlays()
load(reference)
load(result)

#### Choosing a cost
If you have a look at the options, you will notice that there is a cost option to flirt. Indeed, when performing registration, we have a function to measure how well the two images are matching one another. Flirt then attempts transformations to try and improve the fit. This function fit is defined through a cost, among different types.

Which cost should we use? If you were in a pure void, there would be no right or wrong answer from the get-go. No choice but to experiment and find out!

Hopefully, <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FLIRT/UserGuide#flirt">the documentation</a> should give you some pointers. What you want here is to register a T1 to a T1: this is a <u>within</u> modality registration, so you should restrict yourself only to costs appropriate to this type of modality! 

To help you, we've set up a cell that will run the different coregistrations for you. Simply fill in the different costs to consider :)

In [ ]:
possible_costs = ['mutualinfo', 'corratio', 'normcorr', 'normmi', 'leastsq', 'labeldiff']
costs_to_consider = [ ?? ] # fill me with the relevant costs

for c in costs_to_consider:
    flirt(anatomical_target, reference, out=result + '_' + c)


And let's perform lastly the QC step for each of these nice costs. We'll leave it to you to decide if you prefer any cost!

In [ ]:
for c in costs_to_consider:
    load(result + '_' + c)

### Non linear normalization

So, you know how to do it linearly. Non linearly should be easy right?
<i>Well no, it's painfully hard</i>.
To do it, you can use <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FNIRT/">FNIRT</a>. You can browse through, the take-home message is that it is complicated, many steps are involved. 
Out of simplicity, we will not include it in this tutorial.

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>💡 Pay attention! 💡</b></p>
<p style='text-indent: 10px;'>
    FNIRT does NOT expect the input data to be skull-stripped.</p>
</span>
</div>

## Anatomical: conclusions

As a final note, all these steps (<u>including</u> non linear normalization!) can be done automatically for you with a single command: fsl_anat. So you might want to use this command, instead of running all of the above.
Here's a quick run for you (note: it will take several minutes to complete so be patient :) ).

Now, there are some subtleties and renamings that are needed because of the way FSL operates. We thus provide you with a wrapper around fsl_anat to do all this.

In [ ]:
import shutil

def fsl_anat_wrapped(anatomical_target, output_path):
    fsl_anat(img=anatomical_target, clobber=True, nosubcortseg=True, o=output_path)
    # Now move all files from the output_path.anat folder created by FSL to 
    # the actual output_path
    fsl_anat_path = output_path+'.anat'
    files_to_move = glob.glob(op.join(fsl_anat_path, '*'))
    for f in files_to_move:
        shutil.move(f, op.join(output_path, op.split(f)[1]))
    
    # Remove the output_path.anat folder
    os.rmdir(fsl_anat_path)

In [ ]:
fsl_anat_wrapped(anatomical_path, op.join(preproc_root, 'sub-001', 'anat'))

Let's inspect the resulting files:

In [ ]:
print_dir_tree(bids_root, max_depth=5)

That's a lot of files! But let's worry about mostly two of them. <br>
Notice the T1_to_MNI_lin and the T1_to_MNI_nonlin ?
<br>In the former's case, FLIRT was run to obtain a linear normalization, whereas FNIRT was used for the latter to obtain a non linear normalization. But what difference does it make, in practice? Well, let's inspect the results, shall we?

*Hint: consider the brain landmarks, such as the ventricles but also the overall shape of the brain to determine if there was a change and if so which one(s)*

In [ ]:
reset_overlays()
load(reference)
load(op.join(preproc_root, 'sub-001', 'anat', 'T1_to_MNI_lin'))
load(op.join(preproc_root, 'sub-001', 'anat', 'T1_to_MNI_nonlin'))

## 2. fMRI preprocessing

You are now familiar with the few steps of preprocessing revolving around the T1 anatomical file. The main preprocessing starts now, with the functional data. 

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>💡 Do not forget QC! 💡</b></p>
<p style='text-indent: 10px;'>
    As always, in all your steps visualize the effect of what you're doing. This is the easiest way to check that what you're doing is actually having an effect and better yet: a *correct* effect!</p>
</span>
</div>


### 2.0 Problematic volumes removal

A problem can arise in fMRI. To showcase this, please execute the cell below (we'll show you something from another dataset just to drive our point home).

In [ ]:
dataset_demo = 'ds004218'

# Download one subject's data from each dataset
bids_root_demo = op.join(op.dirname(sample.data_path()), dataset_demo)
preproc_root_demo = op.join(bids_root_demo, 'derivatives')
mkdir_no_exist(bids_root_demo)
mkdir_no_exist(op.join(bids_root_demo, 'sub-01'))
mkdir_no_exist(op.join(bids_root_demo, 'sub-01', 'func'))
mkdir_no_exist(preproc_root_demo)
mkdir_no_exist(op.join(preproc_root_demo, 'sub-01'))
mkdir_no_exist(op.join(preproc_root_demo, 'sub-01', 'anat'))
direct_file_download_open_neuro(file_list=['sub-01_task-listening_run-1_bold.nii.gz'],
                                file_types=['func'],
                                dataset_id='ds004218',
                                dataset_version='1.0.0',
                                save_dirs=[op.join(bids_root_demo, 'sub-01', 'func')])

We've downloaded one functional volume from another dataset, because the phenomenon is really visible in this dataset. 
Before going any further in this tutorial, let's open up our data and have a look at them. <u>You should always look at your data before conducting any sort of analysis</u>. See if you find anything at all that looks strange. You should look for

- [ ] Volumes moving in space (ie: head motion)
- [ ] Non homogeneities that do not seem to be coming from brain activity

To open the volume of interest in FSL eyes, simply run:

In [ ]:
load(op.join(bids_root_demo, 'sub-01', 'func', 'sub-01_task-listening_run-1_bold.nii.gz'))

Did you find anything?
If so, what volumes would you remove, approximately?

#### 2.0.1 Field stabilization

The scanner's field takes some time to settle. You probably noticed that the volumes had an initially high contrast that quickly decayed to some baseline? It is precisely caused by the scanner's field settling.
(Add here an image to showcase what might be observed)
There's little to be done in this regard; we can only throw away the volumes that are contaminated in this specific case, as the 'staircase' brain that we observe is not really meaningful and might hurt our analysis later on.

We'll throw away the first 10 volumes (first 20 seconds here), to err on the safe side.

In [ ]:
file_to_realign = glob.glob(op.join(bids_root_demo, 'sub-01', 'func', 'sub-01_task-listening_run-1_bold.nii*'))[0]
output_target = op.join(preproc_root_demo, 'sub-01', 'func', 'sub-01_task-listening_run-1_bold_settled')

# We will start from the 10th volume.
# For this, knowing that there are originally 225 and that you want to throw away the first 10, please fill in
# the following variables
start_vol = 10 # Where should we start? (First volume is 0, not 1 !)
number_of_volumes = 215 # How many volumes should we keep?

fslroi(file_to_realign, output_target, str(start_vol), str(number_of_volumes))

So, take-away message of this section which is also the point of all these preprocessing steps: <u>always look at your data!</u>.
Let's go back to our original dataset now. :)

### 2.1 Field map unwarping

The field itself it not homogeneous, as you've seen in class. This means, in turn, that there are distortions in the acquisition.
We can try to correct for it, through field maps, provided they've been acquired.

Fortunately this is the case in our dataset - but you will need to download them as we have avoided loading them for you - on purpose!

To make sure you've understood how to load datasets, here is the dataset of interest: https://openneuro.org/datasets/ds004226/versions/1.0.0

<img src="imgs/dataset_screen.png">

Your first task is to load:
- Subject 001 fieldmap files located in the fmap subfolder (WITH the JSON sidecars!)

You are free to use either the openneuro cell or resort to the direct download option.
Choose whichever is more convenient for you, download-time wise in particular.

In [ ]:
files_to_include = ['sub-001/fmap/*', # Get here all fmap folder of the subject of interest (INCLUDING THE JSON FILES)
                   ]
openneuro.download(dataset=dataset_fmap, target_dir=bids_root_fmap,
               include=files_to_include, max_concurrent_downloads=1)

In [ ]:
fmap_path = op.join(bids_root, 'sub-001', 'fmap')
mkdir_no_exist(fmap_path)

direct_file_download_open_neuro(file_list=['sub-001_acq-task_dir-AP_epi.nii.gz', 
                                           'sub-001_acq-task_dir-PA_epi.nii.gz',
                                           'sub-001_acq-task_dir-AP_epi.json', 
                                           'sub-001_acq-task_dir-PA_epi.json'],  # Get here all fmap folder of the subject of interest (INCLUDING THE JSON FILES)
                                file_types=['fmap', 'fmap', 'fmap', 'fmap'], 
                                dataset_id=dataset_fmap, 
                                dataset_version='1.0.0', 
                                save_dirs=[fmap_path,
                                           fmap_path,
                                           fmap_path,
                                           fmap_path])

Again, remember this download might not be finished immediately :)
Now, assuming it *is*, let's have a look at what we have:

In [ ]:
print_dir_tree(bids_root_fmap, max_depth=5)

You can see that there are four files, corresponding to two fieldmap acquisitions. One is PA, the other is AP.

We need some parameters to be able to exploit these files.
In particular, we need to figure out:
- The phase encoding direction
- The total readout time

Your task is to figure out which keys to exploit for this.
Have a look at the code below (and feel free to play around a bit of course!) to setup the values properly.
To help you, we've loaded one of the two JSON sidecars:

In [ ]:
get_json_from_file(op.join(fmap_path, 'sub-001_acq-task_dir-{}_epi.json'.format('AP')))

In [ ]:
preproc_fmap_path = op.join(preproc_root, 'sub-001', 'fmap')
direction_file = op.join(preproc_fmap_path, 'datain.txt')

f = open(direction_file, 'w')

for name in ['AP', 'PA']:
    data = get_json_from_file(op.join(fmap_path, 'sub-001_acq-task_dir-{}_epi.json'.format(name)))
    phase_dir = data[???] # Extract here the phase encoding direction !
    total_readout_time = data[???] # Extract here the total readout time !
    
    # We expect a specific format, namely x y z total_readout_time, where x,y and z are set to 1/-1 iff they are the phase
    # encoding direction, 0 otherwise.
    phase = [0, 0, 0, total_readout_time]
    is_neg = len(phase_dir) == 2 and phase_dir[1] == '-'
    phase_dir = phase_dir[0]
    phase[ord(phase_dir)-ord('i')] = -1 if is_neg else 1
    for i in range(3):
        f.write('{} {} {} {}\n'.format(phase[0], phase[1], phase[2], phase[3]))
f.close()

#### Creating the field map
Now, we will create the field map.
This process is tedious, sometimes hard to get right. We've provided you with a function that performs all these steps. Have a look through it and apply it.

In [ ]:
def generate_fmap_AP_PA():
    merged_phase_imgs = op.join(preproc_fmap_path, 'sub-001_acq-task_dir-fmap_merged')
    # Combine AP and PA as single file
    cmd = 'fslmerge -t {} {} {}'.format(merged_phase_imgs,
                                       op.join(fmap_path, 'sub-001_acq-task_dir-AP_epi.nii.gz'),
                                       op.join(fmap_path, 'sub-001_acq-task_dir-PA_epi.nii.gz'))
    os.system(cmd)
    
    # Now we generate the fieldmap
    output_fmap = op.join(preproc_fmap_path, 'fieldmap_ex')
    unwarped_img = op.join(preproc_fmap_path, 'se_epi_unwarped')
    cmd = 'topup --imain={} --datain={} --config={} --fout={} --iout={} -v'.format(
        merged_phase_imgs,
        direction_file, 
        'b02b0.cnf', 
        output_fmap, 
        unwarped_img)
    os.system(cmd)
    
    # Convert fmap units
    fslmaths(output_fmap).mul(6.28).run(output_fmap + '_rads')
    # Create magnitude fmap
    fslmaths(unwarped_img).Tmean().run(output_fmap + '_mag')
    
    # Extract fmap brain using bet
    bet(output_fmap + '_mag', output_fmap + '_mag_brain', robust=True, mask=True)

In [ ]:
generate_fmap_AP_PA()

In the step above, go and check all the steps. In particular, you might want to check first that the generated brain extracted fieldmap is correct and if not correct the brain mask and mask again the fieldmap, as you've done in the anatomical section.
<br><br>

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>💡 Pay attention! 💡</b></p>
<p style='text-indent: 10px;'>
    Easy? Well, not always. Field maps for this dataset came in a specific format. But they can come in *many* different ways, meaning you will need to be very careful when recovering them. The steps outlined above in particular are only applicable in the case of having an AP-PA acquisition. Here is the full resource of FSL's FUGUE on field map unwarping: https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FUGUE/Guide . Don't be afraid to refer to it, should you have a different format in a project!</p>
</span>
</div>


#### Applying the fieldmap

Now that we have our beautiful fieldmap, it remains to apply it.
Note that in general, it is best to apply the fieldmap along with the registration step (to avoid resampling the image too much), but to show you the effect of the fieldmap, let's apply it!

We'll apply it to a single volume, let's say the first volume of our EPI timeseries. Using again fslroi, let's extract it before all things:

In [ ]:
epi_target = op.join(bids_root_fmap, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold')
extracted_epi_path = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_vol_1')
fslroi(epi_target, extracted_epi_path, str(0), str(1))

Now, we just need to figure out some more information and we'll have all we need to use the fieldmap!
We need to know what is called the **effective echo spacing and phase encoding direction of our EPI** (<u>not the fieldmap</u>)!
If you look up in the JSON file, you should find these values in **EffectiveEchoSpacing** and **PhaseEncodingDirection** fields respectively. 
Note that you will likely see phase encoding as being i, j or k. Simply switch to x, y or z respectively. (In practice it's not always so simple, but here it is). 
**In general if you do not know your phase encoding, check the number of slices and contrast with the number of slices you have in every direction. You will figure out then which direction is the phase encoding one.**
Please, fill in the values below based on the JSON file:

In [ ]:
get_json_from_file(epi_target + '.json')

In [ ]:
dwell_time = 0.000620007
unwarpdir= 'y-'

It now remains to apply the fieldmap! Run the command below to unwarp (correct the distortions) based on the fieldmap!

In [ ]:
fmap_path = op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_rads')
result= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_vol_1_fmap')
os.system('fugue -i {} --loadfmap={} --dwell={} --unwarpdir={} -u {}'.format(extracted_epi_path, fmap_path, dwell_time, unwarpdir, result))

Let's visualize!

In [ ]:
reset_overlays()
load(result)
load(extracted_epi_path)

You should observe the following two volumes:

<center>
<div style="display:inline-block;">
    <img src="imgs/uncorrected_brain.png" style="height: 200px;width:auto;border: blue 6px groove;" />
    <p style="text-align:center;">Before unwarp (sagittal)</p>
</div>
<div style="display:inline-block;">
    <img class="middle-img" src="imgs/corrected_brain.png"/ style="height: 200px;width:auto;border: green 6px groove;" />
    <p style="text-align:center;">After unwarp (sagittal)</p>
</div>
<br><br>
<div style="display:inline-block;">
    <img src="imgs/uncorrected_brain_2.png" style="height: 270px;width:250px;border: blue 6px groove;"/>
    <p style="text-align:center;">Before unwarp (axial)</p>
</div>
<div style="display:inline-block;">
    <img src="imgs/corrected_brain_2.png" style="height: 270px;width:250px;border: green 6px groove;" />
    <p style="text-align:center;">After unwarp (axial)</p>
</div>
</center>

Do you think the fieldmap made an improvement? To drive your answer, feel free to inspect both volumes in FSLeyes. Observe the frontal and ventral regions. Do you notice anything different? Which one seems to match better what you'd expect from the brain anatomy ? 


<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b> Assessing quality of functional data 💡</b></p>
<p style='text-indent: 10px;'>
    When in doubt about what you observe, don't be afraid to go have a look at your T1 to compare against. If a structure in the functional shows up in the T1 but distorted, you'll find out faster this way.</p>
</span>
</div>

### Motion correction

Motion correction here specifically means trying to make it such that a given voxel describes the same brain position in all volumes.

To illustrate why it might be a good idea, let's have a look at the functional data of our participant. Watch the movie. Do you notice anything strange?

<center><img src="imgs/motion.gif"/>
    <p style="text-align:center;"><i>You might want to pay attention to the axial view (right)</i></p></center>


The volumes tend to move a bit around, don't they?
<an example moment of motion>
    
This is a problem. Indeed, when we talk of a given voxel, our hope for analysis is that it represents a specific coordinate of anatomy. Imagine if you were trying to find your way with Google Maps, but every now and then the houses would suddenly all move by one kilometer! Would not be so easy to get to the right address, would it? Well, here it's the same. We want that a given (X, Y, Z) position describes always the same portion of the brain, otherwise our analysis will simply not work.

But because of motion, this is not the case.
    
This is one of the core issues of fMRI: the participant simply moved, ever so slightly, during the acquisition. As a consequence, well, we have a recording of a moving participant. This is not a rare phenomenon: imagine having to keep your head perfectly still for several minutes and you'll quickly understand that it is **hard**!
    
Still, we would like to do something about it. This is where motion correction steps in. There are two sides to motion correction. The first, which we'll cover here, attempts to put all volumes back in alignment, so that a given position is indeed consistently describing the same anatomical part for all volumes. The second, which you'll see next week, attempts to correct for the consequences of motion on the magnetic field.
<br><br>
Do you remember Ducky? Well, imagine now that our dear duck has a rare shaking disease.
    <br><img src="imgs/ducky/shakyducky.png" style="width:auto;height:500px;"/>
<br>If we take several consecutive pictures of Ducky, it won't be as aligned as it should be
    <br><img src="imgs/ducky/duckies_before_reg.png" style="width:900px;height:auto;"/>
<br>To correct this, I can apply the idea we used in normalization. Let's pick one Ducky image as reference. Then, all other images of Ducky will be registered to this Ducky
    <br><img src="imgs/ducky/duckies_reg.png" style="width:900px;height:auto;"/>
    <br><img src="imgs/ducky/duckies_after_reg.png" style="width:900px;height:auto;"/>
<br>Now, what if I want to remember how much Ducky had moved ? Well, I can remember the parameters of the transformation I had to apply to align the volumes. This fully encapsulates the motion information.

This is precisely what motion correction sets out to achieve. For this, we need first to define a reference, if possible in fMRI space and that would not require too much transformations. Which option(s) seem reasonable to you?:

- [ ] Choosing as reference a volume of the fMRI timeserie
- [ ] Averaging the fMRI timeserie and using the mean volume as reference
- [ ] Choosing as reference an anatomical volume, preferably of T1 contrast
- [ ] Choosing as reference an fMRI standard space volume, derived from a cohort of participants


It turns out the first two options are usually equivalent. Because it saves us one pass of average computation, we will choose the first option: picking a volume and using it as reference! Which one do you think would be good?
- [ ] The first volume of the timeserie
- [ ] The last volume of the timeserie
- [ ] The middle volume of the timeserie
- [ ] Any volume such that the bold had the time to settle down

Now, let us perform this step, on our **first** dataset (the one without fieldmaps). In FSL, we use <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/MCFLIRT">MCFLIRT</a> to perform this correction.
<br>
    
<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b> 💡 Pay attention ! 💡</b></p>
<p style='text-indent: 10px;'>
    By default, MCFLIRT selects the middle volume of the EPI serie as reference to which other volumes are realigned.</p>
</span>
</div>

In [ ]:
path_original_data = op.join(bids_root_fmap, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold')
path_moco_data = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco')
mcflirt(infile=path_original_data,o=path_moco_data, plots=True, report=True, dof=6, mats=True)

Okay! So, what do we have now?

In [ ]:
print_dir_tree(bids_root_fmap, max_depth=5)

In the functional folder, notice that we have two new files:
```
sub-001_task-sitrep_run-01_bold_moco.nii.gz
sub-001_task-sitrep_run-01_bold_moco.par

```

The first one is the corrected EPI time serie, with volumes realigned. The second is a file describing the motion parameters that were used to move each volume. It will be useful very shortly to determine which volume moved by a lot.
Notice as well a new directory!
```
sub-001_task-sitrep_run-01_bold_moco.mat/
```
This directory is full of .MAT files. These are the transformation matrices used for every volume to realign them.

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b> 💡 Pay attention ! 💡</b></p>
<p style='text-indent: 10px;'>
    The motion parameters and the transformation matrices are related, but they are not exactly the same thing. While you can recover one from the other, it is not trivial. Applying the transformation matrix to a volume will put it 'in alignment' as you've done with FLIRT. However the motion parameters cannot be applied directly. Loosely, the motion parameters describe how you would move if you first applied a rotation along x, then along y, then along y, followed by transition along x, then transition along y, then transition along z. This ordering of transformations is **not** really what happens with the transformation matrices. It is a convention adopted by FSL to make it easier to decouple transformations and rotations in the motion parameter analysis; it is therefore a <b>convenience</b>.
    <u>Do not confuse transformation matrices and motion parameters</u>!</p>
</span>
</div>

Before going <u>any further</u>, go and have a look at the corrected timeseries.

In [ ]:
reset_overlays()
load(path_original_data)
load(path_moco_data)

Did mcflirt help correct motion? Are you convinced it did somewhat a proper job?
<br>
It's actually not too easy to tell right? Well, let's see if we can figure something out to ease our quality control!

#### Motion parameters and degrees of freedom

We told you earlier that motion parameters can be used to estimate the motion along every axis.

In our invocation of mcflirt, notice the following:
```python
mcflirt(..., dof=6)
```
dof stands for <i><b>d</b>egrees <b>o</b>f <b>f</b>reedom</i>, it really means what kind of transformation we wish to apply. In a 3D transformation, we have 3 axis:
<img src="imgs/3d_axis.png"/>
Along each axis, we can apply one transformation. Because we apply here only **affine** transformations, we can choose any transformation from:
- Translation along the axis
- Rotation along the axis
- Shear along the axis
- Scale along the axis

Together, you can see this gives in total **12** DOF.
We've chosen 6 DOFs, which is the standard choice: we want only to translate and rotate around the volumes, since they've been displaced by motion.

#### Looking at the resulting correction parameters
Recall the motion parameters are stored in the .par file produced by MCFLIRT. Notice that since each volume moved differently, we have one transformation per volume, thus one set of motion parameters per volume as well. We provide you with a way to load these parameters:

In [ ]:
def load_mot_params_fsl_6_dof(path):
    return pd.read_csv(path, sep='  ', header=None, 
            engine='python', names=['Rotation x', 'Rotation y', 'Rotation z','Translation x', 'Translation y', 'Translation z'])

mot_params = load_mot_params_fsl_6_dof(op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco.par'))
mot_params

Based on **translation on X alone**, can you find perhaps a volume which exceeds with respect to the **preceding volume** a 0.2 mm displacement?

In [ ]:
# write your code here to inspect quickly the translation on X :)

Some metrics have been created, to compute the displacement of a frame compared to the preceding frame: this is the frame-wise displacement. <br>(see <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3254728/">Power, Jonathan D., et al. "Spurious but systematic correlations in functional connectivity MRI networks arise from subject motion." Neuroimage 59.3 (2012): 2142-2154.</a> for more details).<br>
We can use this one to extract an aggregate measure of motion for all volumes. 

In [ ]:
def compute_FD_power(mot_params):
    framewise_diff = mot_params.diff().iloc[1:]

    rot_params = framewise_diff[['Rotation x', 'Rotation y', 'Rotation z']]
    # Estimating displacement on a 50mm radius sphere
    # To know this one, we can remember the definition of the radian!
    # Indeed, let the radian be theta, the arc length be s and the radius be r.
    # Then theta = s / r
    # We want to determine here s, for a sphere of 50mm radius and knowing theta. Easy enough!
    
    # Another way to think about it is through the line integral along the circle.
    # Integrating from 0 to theta with radius 50 will give you, unsurprisingly, r0 theta.
    converted_rots = rot_params*50
    trans_params = framewise_diff[['Translation x', 'Translation y', 'Translation z']]
    fd = converted_rots.abs().sum(axis=1) + trans_params.abs().sum(axis=1)
    return fd

fd = compute_FD_power(mot_params).to_numpy()

In [ ]:
threshold = np.quantile(fd,0.75) + 1.5*(np.quantile(fd,0.75) - np.quantile(fd,0.25))

In [ ]:
%matplotlib inline
plt.plot(list(range(1, fd.size+1)), fd)
plt.xlabel('Volume')
plt.ylabel('FD displacement (mm)')
plt.hlines(threshold, 0, 370,colors='black', linestyles='dashed', label='FD threshold')
plt.legend()
plt.show()

Okay great, but what if we want to know which volumes are actually above threshold? Simply run the cell below!

In [ ]:
np.where(fd > threshold)[0] + 1

So, you now know which volumes might present motion that is worth checking. Go back to FSLeyes and contrast the uncorrected volumes with the corrected ones. Can you see what sort of motion was problematic and was eliminated?

### Motion-correction: conclusions

Motion correction should always be conducted. As you've seen, it is extremely easy to do and has many benefits. However it is not infaillible. High motion tends to cause non linear effects in the signal that simple motion correction above cannot correct since it has no awareness of the magnetic field. <br>
<br> Motion parameters can, in this case, come to our rescue. As they represent the effect of motion, including them in our modeling to try and correct the signal can help. One could for example include this information in a General Linear Model to regress out the signal of these volumes (censoring) from overall timeseries. ➡️ More on this next week!

### 2.3 Slice time correction

The importance of this step is still investigated in fMRI literature. See <a href="https://www.frontiersin.org/articles/10.3389/fnins.2019.00821/full">here</a> for an in-depth analysis of its impact on the pipeline. One of the take-aways from this reference is that slice-time correction together with motion correction does improve results of fMRI analysis and does not hurt.
Doing it before or after MC is not clear, as you can see in the reference above, so we're *choosing* here to showcase it after motion correction, but only time and further investigations will tell if there's a good order :)


#### 2.3.1- A toy example

To help you understand the underlying theory of slice-time correction, we will start from a rather unreasonable case on synthetic data, which will help you better visualize how slice-time correction affects the patterns.

As you'll see, this step can only be performed if you have knowledge of the way in which slices were acquired. Most of the time, fortunately, this is easy to recover. If it is not present in your data, you can ask the scanner operator to find out which sequence was used for your data acquisition.

Let's get started!

First load the file named "ground_truth.nii" in FSLeyes to visualize it. The data is 4D, go ahead a play a bit around to see exactly what happens during the sequence! (don't forget in case of flickering to untick the "Synchronize movie update" option of FSLeyes).

What you see is the ground truth, ie: the real phenomenon as it plays out!

We now have an acquisition sequence. It performs in slices, but not in a linear order.
Our sequence is even weirder: at a given time, not one but 9 slices are made at the same time! 

In [ ]:
%run generate_smileys.py

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline as Interp
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

def save_array_asnib(array, save_name):
    img = nib.Nifti1Image(array.astype(np.uint8), np.eye(4))
    nib.save(img, save_name)
    
def check_dims(axis_len, seq_len):
    if axis_len != seq_len:
        raise Exception('The number of slices in the sequence is different from the number of slices available in the axis. Are you sure this is the right axis?')

def reslice_with_timings(slice_dir, slice_sequence,input_data, original_times):
    assert(original_times.size == input_data.shape[3])
    
    n_acqs_per_tr = slice_seq.shape[0]
    n_multibands = slice_seq.shape[1]
    n_slices = slice_seq.size
    
    r= -1
    if slice_dir=='x':
        # For each slice in x, interpolate!
        n = input_data.shape[0]
        check_dims(n, n_slices)
        r=0
    elif slice_dir == 'y':
        # For each slice in y, interpolate!
        n = input_data.shape[1]
        check_dims(n, n_slices)
        r=1
    elif slice_dir == 'z':
        # For each slice in z, interpolate!
        n = input_data.shape[2]
        check_dims(n, n_slices)
        r=2    
    else:
        # Undefined yo
        raise Exception('Invalid dimension! Should be x, y or z')
    # Reshape the input data to have r as first dimension!
    input_data = np.swapaxes(input_data, 0, r)
    
    output_data = np.zeros(input_data.shape)
    print(output_data.shape)

    y_s = input_data.shape[1]
    z_s = input_data.shape[2]
    
    # Now, on the first axis, we will iterate over slices :)
    for b in range(0, n_acqs_per_tr):
        time_slice = original_times + b*1./n_acqs_per_tr
        # For all slices acquired together in the multiband
        slices = slice_seq[b]
        print('---------')
        print(time_slice)
        print(slices)
        for s in range(0, n_multibands):
            sl = slices[s]
            for y in range(0, y_s):
                for z in range(0, z_s):
                    lin_interper = Interp(time_slice, input_data[sl, y, z, :], k=1)
                    output_data[sl, y, z, :] = lin_interper(original_times)
    # Now that this very inefficient method is done we should remember to swap back the axis :)
    input_data =np.swapaxes(input_data, r, 0)
    output_data=np.swapaxes(output_data, r, 0)

    output_data[output_data < 0] = 0
    return output_data

In [ ]:
slice_seq = np.arange(0, 99).reshape((11, - 1))

In [ ]:
slice_seq

As you can see, these slices are acquired in a sequential order that can be called either ascending or descending depending on your convention!
It means that the slices are obtained successively.

There are different types of slicing, which depends on your sequence. Notice that in our example we acquire several slices simultaneously (for example, slices 0 up to 9 are all acquired together!). This could be some multiband acquisition, for instance, but really it is mostly to help you visualize the effect of slice timing for the exercise. In practice the slices are defined to sample the signal in the most appropriate way, so our toy sequence will likely be too crude :)

In any case, we had some ground truth signal, that you can visualize in ground_truth_modulation.nii (don't forget to play the movie with the box unticked!)

This signal represents the true signal that we want to acquire.
The participant steps in an MRI, and the scanner operator uses the sequence we've defined above:

```
array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23, 24, 25, 26],
       [27, 28, 29, 30, 31, 32, 33, 34, 35],
       [36, 37, 38, 39, 40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49, 50, 51, 52, 53],
       [54, 55, 56, 57, 58, 59, 60, 61, 62],
       [63, 64, 65, 66, 67, 68, 69, 70, 71],
       [72, 73, 74, 75, 76, 77, 78, 79, 80],
       [81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98]])
```

What this really means is that we acquire 9 slices at the same time and then move on:
(A gif to showcase what this means)

Your task will be two-folds:
1. Understand which axis the sequence was applied on, ie along which direction (X, Y or Z) was slicing performed
2. With this simple knowledge, apply a slice-timing correction algorithm 

This cute smiley will represent our neurological data! Every time point, the behaviour is simple: the smiley's intensity increases across time!
Here is what it looks like in FSLeyes:

In [ ]:
reset_overlays()
load('ground_truth_modulation.nii')
displayCtx.getOpts(overlayList[0]).cmap = 'hot'

In [ ]:
load('ground_truth_upsampled_modulation.nii')
displayCtx.getOpts(overlayList[0]).cmap = 'hot'

Using the following slice sequence, we have acquired our smiley signal.
The resulting timeseries is represented in the acquired_modulation file. 

In [ ]:
reset_overlays()
load('acquired_modulation.nii')
displayCtx.getOpts(overlayList[0]).cmap = 'hot'

Oh no! What went wrong?

Well, if you think about it, nothing. It is simply that acquiring every slice takes time. During this time, the signal is evolving, so we're a little late in our acquisition, which causes the drift you're seeing.

This is what you observe in the acquired modulation file. The slice that is at the top - which is the last slice - also turns out to be the one with highest value. At timestep 0, it is in fact almost equal to 10 - the value of the ground truth at timestep 1 !

#### Correcting the delay

The heart of slice-timing correction is an interpolation in time.
Because the timing of the slices is wrong, we account for it by interpolating back to some reference time. This is how we obtain the resliced data.
For this, we need some informations. The first is the sequence in which slices are acquired, to know the lag of each slice. We also need the axis along which slices are acquired.

Based **only on the abnormal smiley visualization in FSLeyes** and the knowledge of the sequence (that is: a bottom-up sequence with 9 simultaneous bands in every slice), can you determine along with direction the acquisition was made?

- [ ] The X direction
- [ ] The Y direction
- [ ] The Z direction

Based on your answer, in the following cell, fill in the phase_encode variable, with either 'x', 'y' or 'z' (with the quotation marks!).

In [ ]:
phase_encode = 'y'

We will now conduct slice-timing correction: the idea is simply to interpolate back the slices in time along the slice direction. Easy right? Let's do it:

In [ ]:
smile_ts = nib.load('ground_truth_modulation.nii').get_fdata()
smile_resampled = nib.load('acquired_modulation.nii').get_fdata()
resliced_data = reslice_with_timings(phase_encode, slice_seq, smile_resampled, np.arange(0,9))
save_array_asnib(resliced_data.astype(np.uint8), 'resliced_data.nii')

In [ ]:
reset_overlays()
load('resliced_data.nii')
displayCtx.getOpts(overlayList[0]).cmap = 'hot'

Are you convinced on this toy example we did a not-too bad job?

#### 2.3.2  Application to real data

We have shown you the basic principle, but the application to real data requires some specific informations.
You need the following ingredients:
- When was each slice acquired in the sequence: **(Slice timing)**
- Along which axis were the slices acquired: **Phase direction**
- How much time we take to acquire all slices: **TR**

Let's go back to our practical dataset to extract these informations. Can you find them, when looking through the JSON sidecar?

In [ ]:
data = get_json_from_file(op.join(bids_root_demo, 'sub-01', 'func', 'sub-01_task-listening_run-1_bold.json'))
data

This data is actually a dictionary. We can thus extract the slice timing as an array directly from it. For example, to extract TaskName, we would use:
```python
data['TaskName']
```

Go ahead and extract the slice timing array, and store it in the slice_timing variable.

In [ ]:
slice_timing = data['SliceTiming'] # Replace with the appropriate key (have a look above!)

Now, we might want to know where our slices are, ie along which axis, right? Typically it is along the z-direction, but we're better off if we check! Using FSLeyes, determine how many slices each axis has **for the functional data of interest**. You should thus open the relevant functional file in FSLeyes to answer this question.


<div class="warning" style='background-color:#90EE90; color: #112A46; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>Using FSL command line</b></p>
<p style='text-indent: 10px;'>To figure out the dimensions of an MRI image, a faster option - if you have FSL installed directly - is to run the command line command:
    <blockquote>fslhd [your_volume]</blockquote>
This will give you all informations contained within the header of the NIfti file. For example, running the command for our volume will easily allow us to access the slice informations:
    <img src="imgs/fslhd_capture.png"></p>
</span>
</div>
Let's compare now with the amount of slices we have in our acquisition. We can consider simply the number of timings for this

In [ ]:
len(slice_timing)

So we have 42 slices in our slice timings, and you likely found 64 slices on the X axis, 64 axis on the Y axis and 42 slices on the Z axis. As a consequence, Z is the axis where the slices were acquired!
Great, so we know which axis we want, we know the slice timings, but we still need to know the TR. This information is also in the JSON sidecar! Extract it now!

In [ ]:
tr = data['RepetitionTime'] # Extract the TR from the sidecar's appropriate field

To now perform the correction, we need to apply FSL's slicetimer command. For this, we need to save the timings first to their own separate file! Instead of giving the slice timings, we will provide instead the slice **order** (ie which slice was done in which order) and let FSL figure out how to best correct based on this information.

Let's do it.

In [ ]:
slice_order = np.argsort(slice_timing) + 1

# Write to a file the corresponding sorted timings :)
timing_path = op.join(preproc_root_demo,  'sub-01', 'func', 'sub-01_task-listening_run-1_slice-timings.txt')
file = open(timing_path, mode='w')
for t in slice_order:
    file.write(str(t) + '\n')
file.close()

Finally we can call slicetimer from a terminal!

In [ ]:
file_to_realign = op.join(bids_root_demo, 'sub-01', 'func', 'sub-01_task-listening_run-1_bold')
output_target = op.join(preproc_root_demo, 'sub-01', 'func', 'sub-01_task-listening_run-1_bold_slice-corr')
cmd = 'slicetimer -i ' + file_to_realign + ' -o ' + output_target + ' -r ' + str(tr) + ' -d 3 --ocustom=' + timing_path
os.system(cmd)

Had we launched it on the unscrubbed data, we would really notice the impact on the first volume. <span style="color:red;">Notice that you should in general **never** do this, as you would introduce lots of noise and garbage in your data.</span> Prefer to first clean all that is weird and then perform steps that might not bring a visible improvement rather than starting with data that is so bad that you can visually see changes when running above steps.
 <div class="row">
    <img src="imgs/slice_uncorr.png" style="width:100%">
      <center>First volume without slice correction</center>
    <img src="imgs/slice_corr.png" style="width:100%">
       <center>First volume with slice correction: the staircase has been more or less mitigated but the result is still imperfect...</center>
    <center>**And because of the linear interpolation, the garbage of volume 0 was spilled to volume 1!!!**</center>
    <img src="imgs/vol1_slice_uncorr.png" style="width:100%">
      <center>Second volume without slice correction</center>
    <img src="imgs/vol1_slice_corr.png" style="width:100%">
       <center>Second volume with slice correction: the result is worse than before...</center>
</div> 


The message here is: 
- **always** perform QC between your steps
- no algorithm can turn trash to gold. Remove the faulty volumes or you'll likely have a garbage-in garbage-out scenario. 

<u>Remember the 1 - 10 -100 dollar rule! It is much easier to avoid errors than compensate for them.</u>

### Where are we?

So, let's see what we have done so far:

<table>
    <tr><th style='text-align:justify;'>Data type</th><th style='text-align:justify;'>Step name </th><th style='text-align:justify;'>Details of the step</th><th style='text-align:justify;'>FSL tool </th></tr>
    <tr><th>Anatomical</th><td></td><td></td></tr>
    <tr><td></td><td style='text-align:justify;'>Skull stripping</td><td style='text-align:justify;'>Removing skull and surrounding tissues to keep only the brain</td><td style='text-align:justify;'>BET</td></tr>
    <tr><td></td><td style='text-align:justify;'>Segmentation</td><td style='text-align:justify;'>Segmenting brain tissues based on their contrasts</td><td style='text-align:justify;'>FAST</td></tr>
    <tr><td></td><td style='text-align:justify;'>Normalization</td><td style='text-align:justify;'>Mapping participant's brain to a reference brain, making its orientation and scale match so that comparison across participants become feasible.</td><td style='text-align:justify;'>FLIRT</td></tr>
    <tr><th>Functional</th><th></th><th></th></tr>
    <tr><td></td><td style='text-align:left;'>First few volumes removal</td><td style='text-align:justify;'>Removing volumes for which the b0 field is still not stable and that could contaminate all our data if left unchecked.</td><td style='text-align:justify;'>fslroi</td></tr>
    <tr><td></td><td style='text-align:left;'>Field unwarping</td><td style='text-align:justify;'>Correction distortions induced by inhomogeneities of the b0 field through maps acquired specifically to measure this field called fieldmaps.</td><td style='text-align:justify;'>FUGUE (but also FLIRT - see below)</td></tr>
    <tr><td></td><td style='text-align:left;'>Motion correction</td><td style='text-align:justify;'>Realignment of fMRI volumes to a common reference - typically one volume or the average of the volumes - to correct for inter-volume motion. The extracted motion parameters can be used for subsequent analysis (see GLM next week!)</td><td style='text-align:justify;'>MCFLIRT (which is one suboption of FLIRT in fact)</td></tr>
    <tr><td></td><td style='text-align:left;'>Slice-timing correction</td><td style='text-align:justify;'>Accounting for the difference in acquisition between the slices that make up a volume to interpolate back voxels to a fixed time reference.</td><td style='text-align:justify;'>slicetimer</td></tr>
</table>


As mentioned earlier:
- Doing motion correction or slice-timing first is still a matter of debate in the literature
- Field unwarping and coregistration (which you'll see now) can be conducted jointly to improve results. It means that a typical pipeline would actually be in the order: Volumes removal > (Motion correction > slice-timing correction) > Coregistration + Fieldmap unwarping > ...


### Coregistration

Just what is coregistration? Well, it is basically a registration between images of different modalities. In our specific case, we want to register fMRI (EPI) to an anatomical image (T1). There are several reasons for this. The first that comes to mind is that if you overlay your fMRI on the anatomy, you can of course reason much more easily on where you are in the brain, what activations you might be looking at and so forth. Imagine a participant has a brain lesion visible on the anatomy and you want to see how this reflects on the fMRI. Being able to put the two together would make it much easier, would it not?

This is the first reason behind coregistration.

The second is because of normalization. Assume you want to compare all fMRI data of participants. Clearly, putting all of them into a common reference frame is a bit trickier, because of how noisy and low-resolution the data is, right? But you know how to map the anatomical to this common space with excellent accuracy, and you've saved this transformation earlier.
If you could figure out how to go from the fMRI space to anatomical, clearly the problem would be solved! You'd only have then to apply the transformation from anatomical to common space and be done with it.


Computing the fMRI space to anatomical transformation is precisely the goal of coregistration.
<br><br>
To do this step, we will use a wonderful command: <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FLIRT/UserGuide#epi_reg">epi_reg</a> ! As the name states, it is a command to register an EPI. Hard to make it clearer huh? 

#### What to do

Notice that we want to compute the transformation to use for coregistration.
Now, we have an EPI, here of 364 volumes, each supposedly aligned by motion-correction. How many times should we compute the transformation?
- [ ] 364 times, once for each volume
- [ ] Once, selecting any volume from the EPI

Your task is simple. You should:
- Fill in the name of the EPI target. It should be the **motion-corrected** EPI that you corrected using MCFLIRT (ignore the slice-timing corrected volume). If you want to use a single volume, set the use_first_vol variable to True!
- Fill in the path to the whole head T1 image (**before** skull stripping was conducted!)
- Fill in the path to the skull-stripped T1 image (**after** skull stripping was conducted!)

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b> 💡 Pay attention ! 💡</b></p>
<p style='text-indent: 10px;'>
    Make sure that the whole head T1 and the skull-stripped T1 have the same orientation.
For example, if you ran fsl_anat to extract the brain (which is fine), FSL will change in the headers the orientation of the T1 before skull-stripping. As a consequence, the brain-extracted T1 no longer has the same orientation as the original T1. If you display them on top of each other, they are perfectly matched, but not from the perspective of the <b>headers</b>, which can play nasty tricks on you when performing coregistration.</p>
</span>
</div>

In [ ]:
epi_target = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco')
use_first_vol = True
whole_t1 = op.join(preproc_root, 'sub-001', 'anat', 'T1_biascorr')
skull_stripped_t1 = op.join(preproc_root, 'sub-001', 'anat', 'T1_biascorr_brain')
output_path = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_bbr')
ref_vol_name =  op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco_vol_middle')

In [ ]:
if use_single_vol:
    # Extract the middle volume with fslroi as we've seen before :)
    fslroi(epi_target, ref_vol_name, str(182), str(1))
    # Call epi_reg
    epi_reg(ref_vol_name, whole_t1, skull_stripped_t1, output_path)
    # Delete the first volume (we don't need it anymore :0)
else:
    epi_reg(epi_target, whole_t1, skull_stripped_t1, output_path)

Notice how FAST is run?
This is because the specific coregistration cost (boundary-based registration, BBR) uses the anatomical white-matter tissues from FAST. If no such tissue is provided to the function, it re-runs FAST to obtain it and use it. If you've already done anatomical segmentation, clearly there's no need to redo it right?
In particular, imagine if you had to yourself correct the white matter with the help of an expert because somehow FSL did not do a poor job on your data. Clearly you'd like to have this one used instead of the result from FAST, right?

Well- you can! We just need a new option in the epi_reg command:
```python
epi_reg(...,wmseg=path_to_your_white_matter_segmentation)
```

In [ ]:
epi_reg(ref_vol_name, whole_t1, skull_stripped_t1, output_path, wmseg=op.join(preproc_root, 'sub-001', 'anat', 'T1_fast_pve_2'))

Let's overlay the two (EPI and anatomical) on top of each other to visualize the quality of the coregistration!

In [ ]:
reset_overlays()
load(skull_stripped_t1)
load(output_path)

Now, how do we *know* if the registration is good or bad?
Well, there are several things to watch out for, but here are some main leads:
- Is the functional in the right orientation?
- Are the ventricles correctly aligned?
- Are the boundaries of the EPI more or less matching the anatomical?

➡️ You can also check how the white matter of the EPI matches your anatomical's white matter provided you have sufficient resolution

#### Some cleanup
If you have a look, you might notice that perhaps your directory got filled with many files. These are temporary files, created but uncorrectly not eliminated by epi_reg. The following should help:

In [ ]:
def cleanup_epi_reg(path_to_clean):
    patterns = ['*_fast_*', '*_fieldmap*']
    for p in patterns:
        files = glob.glob(op.join(path_to_clean, p))
        for f in files:
            os.remove(f)

In [ ]:
cleanup_epi_reg(op.join(preproc_root, 'sub-001', 'func'))

#### Including the fieldmap correction

If you've looked a bit, you've probably noticed some frontal "horns" in the axial view of the EPI.
While one could hypothesize perhaps the patient had some imp lineage, we notice that these horns are missing on the anatomical. What could explain these horns, based on all you've seen so far?
- [ ] A rare case of astral projection
- [ ] A distortion of the magnetic field

<br> And this is where fieldmaps come into play! For these to work, we need to modify our command in the following ways:
```python
epi_reg(..., wmseg=path_to_wm, fmap=path_to_fmap, fmapmag=path_to_fmap_magnitude, fmapmagbrain=path_to_bet_fmap_magnitude, echospacing=effective_echo_spacing, pedir=pahse_encoding_direction)
```

All these informations have been determined in the field map part of the tutorial, fortunately! Complete the following snippet to fill in all informations and get the coregistration running with fieldmap correction! (Remember: these informations should be extracted from the **EPI**, not the fieldmaps!!)

In [ ]:
output_path_fmap = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_bbr_fmap')
dwell_time = 0.000620007
unwarpdir= 'y-'

In [ ]:
epi_reg(ref_vol_name, 
        whole_t1, 
        skull_stripped_t1, 
        output_path_fmap, 
        wmseg=op.join(preproc_root, 'sub-001', 'anat', 'T1_fast_pve_2'),
        fmap=op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_rads'),
        fmapmag=op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_mag'),
        fmapmagbrain=op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_mag_brain'),
        echospacing=dwell_time,
        pedir=unwarpdir)

Now let's check whether this brought any improvement at all!

In [ ]:
load(output_path_fmap)

What do you think? Do you think it is any better? If so why, if so why not?

NOTE:
epi_reg is still using FLIRT under the hood! To be more specific, it is using flirt setup with its search cost as BBR. If you look through FLIRT's options, you'll notice that many more options are open to you:
<img src="imgs/flirt_options.png"/>

Feel free to explore the effect of different search costs :) But remember: not all costs are born equal when registering images **across modalities**!

## Applying the transformation to all volumes

You must have noticed that the coregistration has been done only on a single EPI volume, right?
This is, as you've guessed, because all volumes are aligned with respect to each other following motion-correction. As such, it is wasteful to compute more than once the transformation from EPI to anatomical.

The order of transformations we would like to have is:

<img src="imgs/transforms.png"/>

Notice that motion correction was applied by selecting a reference volume in the EPI. By default, the middle EPI was used. So, let's extract this EPI. It will be on this one that we will work, compute all transformations and reason.
**It is critical that you pay attention to which image was used to compute your transformations, otherwise combining them won't make sense!**.
For this reason, let's now go over the entire pipeline and transformation steps, sticking to this middle EPI. We extract it again with fslroi, as you've seen before:

In [ ]:
original_epi = op.join(bids_root_fmap, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold')
middle_epi = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_middle-vol')
fslroi(original_epi, middle_epi, str(182), str(1))

Now, let's do motion correction. Recall that it is done on the **entire** EPI timeseries with mcflirt. We will explicitly give the middle epi as reference this time around, to force FSL to use this volume and realign everyone to it!

In [ ]:
path_moco_data = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco')
mcflirt(infile=original_epi,o=path_moco_data, plots=True, report=True, dof=6, mats=True, reffile=middle_epi)

Fantastic! Now, because **the reference volume did not move at all** (since it is the reference to which everyone is realigned), we can use this volume as starting point to compute our other transforms: we're only missing the coregistration with fieldmap unwarping, as the normalization is obtained through the anatomical data :)

In [ ]:
whole_t1 = op.join(preproc_root, 'sub-001', 'anat', 'T1_biascorr')
skull_stripped_t1 = op.join(preproc_root, 'sub-001', 'anat', 'T1_biascorr_brain')
output_path = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_anat-space')
dwell_time = 0.000620007
unwarpdir='y-'

epi_reg(middle_epi, 
        whole_t1, 
        skull_stripped_t1, 
        output_path, 
        wmseg=op.join(preproc_root, 'sub-001', 'anat', 'T1_fast_pve_2'),
        fmap=op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_rads'),
        fmapmag=op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_mag'),
        fmapmagbrain=op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_mag_brain'),
        echospacing=dwell_time,
        pedir=unwarpdir)

Beautiful! 
➡️ Inspect the two resulting files to ensure that nothing went wrong. In other words:
- Check that the MCFLIRT result is okay with respect to motion
- Check that the realignment following epi_reg made the EPI well aligned with the anatomical

Now, if you inspect the folder you'll see we have several files. Some of them are...Well garbage left-over by FSL, but others are more precious: they are transformation files, which describe the transforms we just did.
Namely:

--
```
sub-001_task-sitrep_run-01_bold_bbr_fmap_warp
```
corresponds to the anatomical + field unwarping transformation.
The \_warp was added by FSL, to signal that this is a nonlinear transformation (otherwise it would be a .mat file, but here this is a NIFTI!). But what sort of transformation is it? Where does it map?
It actually goes from EPI (after motion correction) to Anatomical space.
To convince yourself of this fact, you can apply the transformation yourself, using the applywarp function, whose signature is as follows:
```python
applywarp(volume_on_which_transform_is_applied, reference_volume, warp1=transformation_to_apply)
```
--
```
sub-001_task-sitrep_run-01_bold_bbr_fmap_warp
```
corresponds to the motion correction transforms we had (one per each volume!)


<u>Our goal</u>: now that we've computed all these transformations **for our reference volume**, we will combine them and apply all of them at every volume of the EPI!
This way, we will perform an interpolation of our data exactly once, after the last transform and thus minimize error propagation.

However we must tackle one problem: FSL provides us with a tool to apply transformations (even non linear ones such as fieldmap correction) per volume, but not in 4D.
We will thus:
- Split our 4D volume in individual 3D volumes
- Apply the transformation corresponding to each volume separately
- Combine back the volumes in a single 4D volume
- Remove intermediary volumes

The first step, to make everything clear, is to work on the transformations. Let's get started!


### Which transformations to combine?
    
Answer the following question as brain teaser:

- [ ] Knowing the MCFLIRT transform uses FLIRT and is done to perform motion-correction with 6 dof, is it a linear transformation represented by a matrix or a non linear transformation represented by a NIFTI volume?
- [ ] Assuming we have a total transformation EPI -> Standard space, should our reference be the anatomical volume in standard space or the starting EPI?
- [ ] Do we need one individual EPI (**before** moco) > Standard space transform per EPI volume?

Okay, now we will combine the three transformations as a single warp. To do so, we use the following code snippet:

In [ ]:
def combine_all_transforms(reference_volume, warp_save_name,  is_linear, epi_2_moco=None, epi_2_anat_warp=None, anat_2_standard_warp=None):
    """
    Combines transformation before motion correction all the way to standard space transformation
    The various transformation steps are optional. As such, the final warp to compute is based on 
    which transforms are provided.
    """
    args_base = {'premat': epi_2_moco, 'warp1': epi_2_anat_warp}
    if is_linear:
        args_base['postmat'] = anat_2_standard_warp
    else:
        args_base['warp2'] = anat_2_standard_warp
    args_filtered = {k: v for k, v in args_base.items() if v is not None}

    #print(args_filtered)
    convertwarp(warp_save_name, reference_volume, **args_filtered)

Now, to apply the transformation:

In [ ]:
def apply_transform(reference_volume, target_volume, output_name, transform):
    applywarp(target_volume,reference_volume, output_name, w=transform, rel=True)

Using these two functions should not be too hard now. Notice that in combine_all_transforms, setting any transform to None instead of the correct transform will skip the transform step in the total transformation. This way, you should be able to perform quality control. In particular, please ensure that:
- [ ] Applying ONLY motion correction transformation to the first volume yields the expected alignement (so it should be aligned with the \_moco volume.)
- [ ] Applying motion correction + epi -> anat should be aligned to anatomical
- [ ] Finally, applying motion correction + epi > anat + anat > standard should be aligned to the standard
Only once you're convinced these steps are working well should you proceed to standard space. **Remember the 1-10-100 rule! Always perform QC before moving on.**

To help you, we provide you below with the template to do such a thing, so that you don't have to worry too much about the nitty gritty details.
Focus on:
- The reference file to use 
- The transformations to provide (either a file or None)

<b>Notice this is performed only on a single volume. Indeed, if you are debugging you should avoid wasting time applying transformations on entire timeseries to quickly diagnose whether a step is working or failing.</b>

In [ ]:
import time
ref=op.join(preproc_root, 'sub-001', 'anat', 'T1_to_MNI_nonlin.nii.gz')
anat_2_mni= op.join(preproc_root, 'sub-001', 'anat', 'T1_to_MNI_nonlin_coeff.nii.gz')
func_2_anat= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_bbr_fmap_warp.nii.gz')

# We show this one when selecting the middle EPI (volume 182)
middle_epi = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_middle-vol')
split_nbr = '0182'

# We will name its warp as split0182 to show you how to do this for any volume
warp_name = op.join(preproc_root, 'sub-001', 'func', 'sub-001_split' + split_nbr + '_epi_2_std_warp')

# Get the transformation matrix of this volume (this one is actually the unit matrix, 
# since this volume is the reference)
epi_moco = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco.mat/', 'MAT_' + split_nbr)

s0 = time.time()

# -- Step 1: Combine the transformations, that is :
#    EPI -> Motion correction -> Coregistration to anatomical -> Normalization to standard
#    We obtain a single warp, going from EPI -> Standard space
combine_all_transforms(ref, warp_name,  False, epi_2_moco=epi_moco, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni)
s1 = time.time()
out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr)

# -- Step 2: Apply the transformation to our EPI
applywarp(middle_epi,ref, out_vol, w=warp_name, rel=True)
s2 = time.time()

Above, we've timed the steps to estimate which one might be more expensive, between combining the transforms and applying them. Let's check:

In [ ]:
print('Transform combination time:', s1 - s0)
print('Apply transform time:', s2 - s1)

As you can clearly see, combining the transforms is more than 100 times slower than applying the final transform. As a consequence, we would like to do this step as rarely as we can.

#### Optimizing a bit

Okay, so this step is slow. Can we make it faster? Well, yes!

Note that computing all these non linear fields <u>will</u> take time. We've seen above in fact that it is <u>the</u> most expensive step.
Now, applywarp has a neat option. It allows us to apply a transformation using a pre transformation matrix followed by the warp. Why is it cool?
Well, remember this picture?

What if we made it this way?

Then, we only compute one warp once, and we can call applywarp on the volumes themselves, which will be much quicker and is **strictly** equivalent! To do this, we must combine all transforms except the motion-correction transform, which is easily done:
```python
combine_all_transforms(..., epi_2_moco=None,...)
applywarp(split_vol, ref, out_vol, w=warp_name, rel=True, premat=epi_moco)
```

Let's compare the two methods, runtime wise:

In [ ]:
# ----- START OF METHOD 1 
# In this method, we compute the transform from start to finish and apply it
s0 = time.time()
combine_all_transforms(ref, warp_name,  False, epi_2_moco=epi_moco, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni)
s1 = time.time()
out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr + '_v1')
applywarp(middle_epi,ref, out_vol, w=warp_name, rel=True)
s2 = time.time()
# ----- START OF METHOD 2
# In this method, we compute the transform only post motion correction. We apply the motion correction and then the warp
combine_all_transforms(ref, warp_name,  False, epi_2_moco=None, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni)
s3 = time.time()
out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr + '_v2')
applywarp(middle_epi,ref, out_vol, w=warp_name, premat=epi_moco, rel=True)
s4 = time.time()

print('Method 1 runtime:', s2 - s0, '({} for combination, {} to apply)'.format(s1 - s0, s2 - s1))
print('Method 2 runtime:', s4 - s2, '({} for combination, {} to apply)'.format(s3 - s2, s4 - s3))

To convince you that the two produced images are almost identical (you might notice differences on the order of the $10^{-3}$, but consider the relative error this entails and why such an error might happen):

In [ ]:
reset_overlays()
load(op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr + '_v1'))
load(op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr + '_v2'))

Why does it matter? Well, just applying a back-of the envelope calculation, the first method takes 40s per volume, while the second method takes 25 seconds to combine **once** the transforms excluding motion correction, and 0.4 seconds per volume to apply the transforms including motion correction. If we plot the two with an increasing number of volumes, we can see why this quickly becomes relevant:

In [ ]:
%matplotlib inline
x = np.arange(0, 1000, 10)
plt.plot(x, x*40, label='Method 1')
plt.plot(x, 25 + x*0.3, label='Method 2')
plt.xlabel('Number of volumes')
plt.ylabel('Runtime (seconds) [LOG SCALE]')
plt.legend()
plt.yscale('log')
plt.show()

Hopefully, you're convinced that:
- We don't lose anything using method 2 imaging-wise
- We have a benefit in using method 2, computation-wise


### Applying it to the entire timeseries

With all this in mind, let's now apply our transformation to all our volumes! The steps are:

1. Split our EPI into all individual volumes (remember: applywarp only works on a single 3D image but our EPI is 4D).
2. Combine all transformations from EPI after motion correction all the way to standard space **once**. 
3. Use applywarp for every volume, passing the motion correction transform of this volume and the EPI > standard space warp
4. Combine back all volumes as a single 4D EPI in standard space

Ready? Let's go!

In [ ]:
# We will split our starting EPI volume across time 
split_target = original_epi
split_name = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_split')
cmd = 'fslsplit ' + split_target + ' ' + split_name + ' -t'

os.system(cmd)

In [ ]:
# Now, let's combine the different transforms EXCEPT motion correction!
ref=op.join(preproc_root, 'sub-001', 'anat', 'T1_to_MNI_nonlin')
anat_2_mni= op.join(preproc_root, 'sub-001', 'anat', 'T1_to_MNI_nonlin_coeff')
func_2_anat= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_anat-space_warp')
warp_name = op.join(preproc_root, 'sub-001', 'func', 'sub-001_epi_moco_2_std_warp')

combine_all_transforms(ref, warp_name,  False, epi_2_moco=None, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni)

# Here, we will apply transformation WITH motion correction to all our volumes
produced_vols = []

# Notice that we are sorting the volumes here! This is important, to make sure we don't get them in random order :)
split_vols = sorted(glob.glob(op.join(preproc_root, 'sub-001', 'func', '*_bold_split*')))
for split_vol in split_vols:
    split_nbr = split_vol.split('_')[-1].split('.')[0].split('split')[1]
    epi_moco = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco.mat/', 'MAT_' + split_nbr)
    out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr)
    applywarp(split_vol,ref, out_vol, w=warp_name, premat=epi_moco, rel=True)
    produced_vols.append(out_vol)

In [ ]:
total_epi = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std')
tr = 1.5
cmd = 'fslmerge -tr ' + total_epi + ' ' + ' '.join(produced_vols) + str(tr)
os.system(cmd)

# Let's not forget to remove all the temporary volumes, shall we?
for v in split_vols + produced_vols:
    l = glob.glob(v + '*')
    for e in l:
        os.remove(e)

Now, let's check we did a proper job. If we did a proper mapping, we should definitely observe the EPI positioned on the anatomical in MNI space. How well did we do?

In [ ]:
reset_overlays()
load(ref)
load(total_epi)

## Smoothing
All these transforms are not exactly perfect. As you've seen in class, a step of smoothing is typically applied, with the size of the smoothing being dependent on your application, starting resolution etc.
The idea of smoothing is really that, as you're averaging, hopefully you increase the signal to noise ratio. <br>
A side-effect is that finest patterns of activation will be lost in the averaging (we can't have everything: there's no free lunch).

With FSL, smoothing rather easy to do! Simply run the following command :)

In [ ]:
cmd = 'fslmaths ' + total_epi + ' -kernel gauss 6 ' + total_epi + '_smoothed-6mm'
os.system(cmd)

Let's observe what we have now:

In [ ]:
load(total_epi + '_smoothed-6mm')

Do you feel as though the signal-to-noise ratio was improved?

## MRI + fMRI preprocessing: summary

Congratulations! You have reached the end of the MRI + fMRI tutorial on preprocessing!

You should know by now: preprocessing is extremely important and you will likely spend a lot of time on it. Decisions in preprocessing will affect your analysis, so do not take this step lightly, it is <u>critical</u> to do it as well as possible!

<u>Always perform quality control to ensure everything is okay!</u>

Let's review one last time the different steps you've studiedm and which FSL tool(s) can be used to do it:
<table>
    <tr><th style='text-align:justify;'>Data type</th><th style='text-align:justify;'>Step name </th><th style='text-align:justify;'>Details of the step</th><th style='text-align:justify;'>FSL tool </th></tr>
    <tr><th>Anatomical</th><td></td><td></td></tr>
    <tr><td></td><td style='text-align:justify;'>Skull stripping</td><td style='text-align:justify;'>Removing skull and surrounding tissues to keep only the brain</td><td style='text-align:justify;'>BET</td></tr>
    <tr><td></td><td style='text-align:justify;'>Segmentation</td><td style='text-align:justify;'>Segmenting brain tissues based on their contrasts</td><td style='text-align:justify;'>FAST</td></tr>
    <tr><td></td><td style='text-align:justify;'>Normalization</td><td style='text-align:justify;'>Mapping participant's brain to a reference brain, making its orientation and scale match so that comparison across participants become feasible.</td><td style='text-align:justify;'>FLIRT + FNIRT</td></tr>
    <tr><th>Functional</th><th></th><th></th></tr>
    <tr><td></td><td style='text-align:left;'>First few volumes removal</td><td style='text-align:justify;'>Removing volumes for which the b0 field is still not stable and that could contaminate all our data if left unchecked.</td><td style='text-align:justify;'>fslroi</td></tr>
    <tr><td></td><td style='text-align:left;'>Field unwarping</td><td style='text-align:justify;'>Correction distortions induced by inhomogeneities of the b0 field through maps acquired specifically to measure this field called fieldmaps.</td><td style='text-align:justify;'>FUGUE (but also FLIRT when combined with coregistration)</td></tr>
    <tr><td></td><td style='text-align:left;'>Motion correction</td><td style='text-align:justify;'>Realignment of fMRI volumes to a common reference - typically one volume or the average of the volumes - to correct for inter-volume motion. The extracted motion parameters can be used for subsequent analysis (see GLM next week!)</td><td style='text-align:justify;'>MCFLIRT (which is one suboption of FLIRT in fact)</td></tr>
    <tr><td></td><td style='text-align:left;'>Slice-timing correction</td><td style='text-align:justify;'>Accounting for the difference in acquisition between the slices that make up a volume to interpolate back voxels to a fixed time reference.</td><td style='text-align:justify;'>slicetimer</td></tr>
    <tr><td></td><td style='text-align:left;'>Coregistration to anatomical</td><td style='text-align:justify;'>Putting the functional volumes in anatomical space</td><td style='text-align:justify;'>FLIRT (epi_reg being a specialized instance)</td></tr>
    <tr><td></td><td style='text-align:left;'>Smoothing</td><td style='text-align:justify;'>Allowing a bit of lee-way in the voxel's values to account for the imperfection of the registration</td><td style='text-align:justify;'>fslmath with smoothing operation</td></tr>
</table>

You are done! Congratulations!